In [1]:
# 환경 변수 로드
from dotenv import load_dotenv

load_dotenv()

True

In [2]:
from dataclasses import dataclass
# 데이터 정의
## 검색어
data = "중국"

## 검색될 내용
@dataclass
class SearchResult:
  keyword: str
  title: str
  link: str
  content: str


In [3]:
# 임시 데이터
datas: list[SearchResult] = []
datas.append(SearchResult(
  keyword=data,
  title=""""중국 손님 안 받아요" 문구에 '발칵'…구청장 나섰다 [소셜픽]""",
  content="""서울 성동구의 한 카페가 인종차별 논란에 휩싸였습니다.

중국인 손님을 받지 않겠다고 내걸면섭니다.

이 카페의 인스타그램 소개 문구입니다.

영어로 "중국인 손님을 받지 않습니다"라며 우는 얼굴 이모티콘을 달았습니다.

중국에선 이 카페에 갔다가 쫓겨났다는 내용이 퍼지기도 했습니다.

[헨리/영상 크리에이터 : 여기 한국에서 본 카페 중에 가장 인종차별적인 카페예요. 이해가 안 돼요. 왜 내 나라를 이렇게 증오하는지, 누가 이렇게 사업하라고 가르쳤나요?]

소셜미디어를 중심으로 인종차별적이라는 비판이 잇따랐는데, 제재를 해야 한다는 의견에 카페가 위치한 성동구의 자치단체장도 답을 남겼습니다.

정원오 구청장은 "보내주신 우려의 마음을 저 또한 깊이 공감한다"며 "최대한 해당 업장을 설득해볼 수 있도록 노력하겠다"고 밝혔습니다.

이웃나라 일본에서도 비슷한 사건이 있었습니다.

지난해 7월, 도쿄 오오쿠보에 있는 한 레스토랑에서는 "다양성이나 관용으로 말이 많지만 그럴 마음이 없기 때문에 중국인과 한국인은 사절합니다"고 적어 논란이 되기도 했습니다.

[화면출처 인스타그램 'henryli4340']
정재우 앵커 (jung.jaewoo@jtbc.co.kr)""",
  link="https://n.news.naver.com/article/437/0000462099?cds=news_media_pc"
))

In [4]:
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain.schema import Document

@dataclass
class Metadata:
  keyword: str
  link: str

contents: list[str] = [("# " + data.title + "\n" + data.content) for data in datas]
metadatas: list[Metadata] = [
  Metadata(
    data.keyword,
    data.link
  ) for data in datas
]

In [5]:
# 스플리터
# 문서 청크로 나누기
splitter = RecursiveCharacterTextSplitter(
  chunk_size=300,
  chunk_overlap=50
)

docs: list[Document] = []

for i in range(len(contents)):
  splits = splitter.create_documents(
    texts=[contents[i]],
    metadatas=[{
      "keyword":metadatas[i].keyword,
      "link":metadatas[i].link,
    }],
  )
  docs.extend(splits)
len(docs)

3

In [ ]:
# vstore 생성
from langchain_upstage import UpstageEmbeddings
from langchain_chroma import Chroma

# 모델 선정
vstore = Chroma(
  collection_name="knowledge_base",
  embedding_function=UpstageEmbeddings(model="embedding-query"),
  persist_directory="./chroma"
)

In [7]:
# vstore에 문서 삽입
vstore.add_documents(docs)

['90aa63dd-6f3f-4523-b2df-3e9e96b2d6c0',
 '883f5066-d4e5-4786-9983-50cc74bac50a',
 '880a7918-5ec4-4d20-b6c9-6a9ae0721006']

In [ ]:
# 프롬프트
search_prompt="이 검색어는 해당 내용과 관련되어있는 것으로 보입니다. 관련이 있을 경우 참고하고, 아닌 것 같으면 무시하십시오: "

# 검색자
retriever=vstore.as_retriever(
  search_type="mmr",
  search_kwargs={
    "k":4,
    "fetch_k":20,
    "lambda_mult":0.5
  }
)

# 검색
## mmr로만 검색했을 경우

In [16]:
search_results = retriever.get_relevant_documents(data)

prompt = search_prompt
for result in search_results:
  prompt += ("\n" + result.page_content)

print(prompt)

이 검색어는 해당 내용과 관련되어있는 것으로 보입니다. 관련이 있을 경우 참고하고, 아닌 것 같으면 무시하십시오: 
# "중국 손님 안 받아요" 문구에 '발칵'…구청장 나섰다 [소셜픽]
서울 성동구의 한 카페가 인종차별 논란에 휩싸였습니다.

중국인 손님을 받지 않겠다고 내걸면섭니다.

이 카페의 인스타그램 소개 문구입니다.

영어로 "중국인 손님을 받지 않습니다"라며 우는 얼굴 이모티콘을 달았습니다.

중국에선 이 카페에 갔다가 쫓겨났다는 내용이 퍼지기도 했습니다.

[헨리/영상 크리에이터 : 여기 한국에서 본 카페 중에 가장 인종차별적인 카페예요. 이해가 안 돼요. 왜 내 나라를 이렇게 증오하는지, 누가 이렇게 사업하라고 가르쳤나요?]
소셜미디어를 중심으로 인종차별적이라는 비판이 잇따랐는데, 제재를 해야 한다는 의견에 카페가 위치한 성동구의 자치단체장도 답을 남겼습니다.

정원오 구청장은 "보내주신 우려의 마음을 저 또한 깊이 공감한다"며 "최대한 해당 업장을 설득해볼 수 있도록 노력하겠다"고 밝혔습니다.

이웃나라 일본에서도 비슷한 사건이 있었습니다.

지난해 7월, 도쿄 오오쿠보에 있는 한 레스토랑에서는 "다양성이나 관용으로 말이 많지만 그럴 마음이 없기 때문에 중국인과 한국인은 사절합니다"고 적어 논란이 되기도 했습니다.
[화면출처 인스타그램 'henryli4340']
정재우 앵커 (jung.jaewoo@jtbc.co.kr)


## similarity랑 같이 써서 관련 없는건 컷할 경우

In [ ]:
data = "중국"
candidates = retriever.get_relevant_documents(data)
scored = vstore.similarity_search_with_score(data, k=20)

THRESHOLD = 1.55
filtered = [
    doc for doc, score in scored if score <= THRESHOLD
]

# scored
filtered

[(Document(id='90aa63dd-6f3f-4523-b2df-3e9e96b2d6c0', metadata={'keyword': '중국', 'link': 'https://n.news.naver.com/article/437/0000462099?cds=news_media_pc'}, page_content='# "중국 손님 안 받아요" 문구에 \'발칵\'…구청장 나섰다 [소셜픽]\n서울 성동구의 한 카페가 인종차별 논란에 휩싸였습니다.\n\n중국인 손님을 받지 않겠다고 내걸면섭니다.\n\n이 카페의 인스타그램 소개 문구입니다.\n\n영어로 "중국인 손님을 받지 않습니다"라며 우는 얼굴 이모티콘을 달았습니다.\n\n중국에선 이 카페에 갔다가 쫓겨났다는 내용이 퍼지기도 했습니다.\n\n[헨리/영상 크리에이터 : 여기 한국에서 본 카페 중에 가장 인종차별적인 카페예요. 이해가 안 돼요. 왜 내 나라를 이렇게 증오하는지, 누가 이렇게 사업하라고 가르쳤나요?]'),
  1.4655797481536865),
 (Document(id='883f5066-d4e5-4786-9983-50cc74bac50a', metadata={'keyword': '중국', 'link': 'https://n.news.naver.com/article/437/0000462099?cds=news_media_pc'}, page_content='소셜미디어를 중심으로 인종차별적이라는 비판이 잇따랐는데, 제재를 해야 한다는 의견에 카페가 위치한 성동구의 자치단체장도 답을 남겼습니다.\n\n정원오 구청장은 "보내주신 우려의 마음을 저 또한 깊이 공감한다"며 "최대한 해당 업장을 설득해볼 수 있도록 노력하겠다"고 밝혔습니다.\n\n이웃나라 일본에서도 비슷한 사건이 있었습니다.\n\n지난해 7월, 도쿄 오오쿠보에 있는 한 레스토랑에서는 "다양성이나 관용으로 말이 많지만 그럴 마음이 없기 때문에 중국인과 한국인은 사절합니다"고 적어 논란이 되기도 했습니다.'),
  1.5356216430664062),
 (Docu

In [17]:
search_results

[Document(id='90aa63dd-6f3f-4523-b2df-3e9e96b2d6c0', metadata={'keyword': '중국', 'link': 'https://n.news.naver.com/article/437/0000462099?cds=news_media_pc'}, page_content='# "중국 손님 안 받아요" 문구에 \'발칵\'…구청장 나섰다 [소셜픽]\n서울 성동구의 한 카페가 인종차별 논란에 휩싸였습니다.\n\n중국인 손님을 받지 않겠다고 내걸면섭니다.\n\n이 카페의 인스타그램 소개 문구입니다.\n\n영어로 "중국인 손님을 받지 않습니다"라며 우는 얼굴 이모티콘을 달았습니다.\n\n중국에선 이 카페에 갔다가 쫓겨났다는 내용이 퍼지기도 했습니다.\n\n[헨리/영상 크리에이터 : 여기 한국에서 본 카페 중에 가장 인종차별적인 카페예요. 이해가 안 돼요. 왜 내 나라를 이렇게 증오하는지, 누가 이렇게 사업하라고 가르쳤나요?]'),
 Document(id='883f5066-d4e5-4786-9983-50cc74bac50a', metadata={'keyword': '중국', 'link': 'https://n.news.naver.com/article/437/0000462099?cds=news_media_pc'}, page_content='소셜미디어를 중심으로 인종차별적이라는 비판이 잇따랐는데, 제재를 해야 한다는 의견에 카페가 위치한 성동구의 자치단체장도 답을 남겼습니다.\n\n정원오 구청장은 "보내주신 우려의 마음을 저 또한 깊이 공감한다"며 "최대한 해당 업장을 설득해볼 수 있도록 노력하겠다"고 밝혔습니다.\n\n이웃나라 일본에서도 비슷한 사건이 있었습니다.\n\n지난해 7월, 도쿄 오오쿠보에 있는 한 레스토랑에서는 "다양성이나 관용으로 말이 많지만 그럴 마음이 없기 때문에 중국인과 한국인은 사절합니다"고 적어 논란이 되기도 했습니다.'),
 Document(id='880a7918-5ec4-4d20-b6c9-6a9ae0721006', m

In [ ]:
THRESHOLD = 0.75  # 예시. 임베딩 모델에 따라 조정 필요

filtered = []
for doc in candidates:
    # 점수를 lookup해서 붙이는 방식 필요 (id 기준 등)
    score = score_lookup(doc)  # 구현: doc.metadata["link"] 등으로 매칭
    if score >= THRESHOLD:
        filtered.append(doc)

final_docs = filtered